In [99]:

"""
This is an upgraded version of Ceshine's LGBM starter script, simply adding
more average features and weekly average features on it.
"""
from datetime import date, timedelta

import pandas as pd
import numpy as np
import lightgbm as lgb
import sys
import math
import gc
import sklearn.metrics as skl_metrics
from sklearn.metrics import mean_squared_error

# import math
# import sklearn.metrics as skl_metrics
# from sklearn.metrics import mean_squared_error

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger

logger = getLogger(__name__)

pd.options.mode.chained_assignment = None  # default='warn'

DIR = '../logs/'

log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
handler = StreamHandler()
handler.setLevel('INFO')
handler.setFormatter(log_fmt)
logger.addHandler(handler)

handler = FileHandler(DIR + 'train.py.log', 'a')
handler.setLevel(DEBUG)
handler.setFormatter(log_fmt)
logger.setLevel(DEBUG)
logger.addHandler(handler)

logger.info('start')

##########################################################################

train_out = pd.read_pickle('../data/T40_train_storeitem_1s.p')
val_out = pd.read_pickle('../data/T40_val_storeitem_1s.p')
X_test_out = pd.read_pickle('../data/T40_test_storeitem_1s.p')


df_test = pd.read_csv(
    "../input/test_1s.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)
    
items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items = items.reindex(train_out.item_nbr)

items_val = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")
items_val = items_val.reindex(val_out['item_nbr'])

logger.info('Load data successful')

2018-01-03 11:54:32,079 __main__ 41 [INFO]    [<module>] start 
2018-01-03 11:54:32,079 __main__ 41 [INFO]    [<module>] start 
2018-01-03 11:54:32,079 __main__ 41 [INFO]    [<module>] start 
2018-01-03 11:54:32,079 __main__ 41 [INFO]    [<module>] start 
2018-01-03 11:54:32,079 __main__ 41 [INFO]    [<module>] start 
2018-01-03 11:54:32,079 __main__ 41 [INFO]    [<module>] start 
2018-01-03 11:54:32,202 __main__ 68 [INFO]    [<module>] Load data successful 
2018-01-03 11:54:32,202 __main__ 68 [INFO]    [<module>] Load data successful 
2018-01-03 11:54:32,202 __main__ 68 [INFO]    [<module>] Load data successful 
2018-01-03 11:54:32,202 __main__ 68 [INFO]    [<module>] Load data successful 
2018-01-03 11:54:32,202 __main__ 68 [INFO]    [<module>] Load data successful 
2018-01-03 11:54:32,202 __main__ 68 [INFO]    [<module>] Load data successful 


In [49]:
X_test_out.tail(1)

,date,day_1_2017,item_nbr,mean_140_2017,mean_14_2017,mean_182_2017,mean_21_2017,mean_30_2017,mean_364_2017,mean_3_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
3550,2017-08-16,0.0,2127114,0.004951,0.049511,0.003809,0.033007,0.023105,0.001904,0.0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
val_out.head(1)

,index,date,day_1_2017,item_nbr,mean_140_2017,mean_14_2017,mean_182_2017,mean_21_2017,mean_30_2017,mean_364_2017,...,day7,day8,day9,day10,day11,day12,day13,day14,day15,day16
0,0,2017-07-26,0.0,96995,0.112661,0.177493,0.086662,0.118329,0.105935,0.043331,...,0.0,0.693147,1.098612,0.0,0.0,1.098612,1.098612,0.0,0.0,0.693147


In [51]:
train_out.head(1)

,index,date,day_1_2017,item_nbr,mean_140_2017,mean_14_2017,mean_182_2017,mean_21_2017,mean_30_2017,mean_364_2017,...,day7,day8,day9,day10,day11,day12,day13,day14,day15,day16
0,0,2016-08-03,0.0,96995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
logger.info('Preparing traing dataset...')

all_columns = train_out.columns.tolist()

y_columns = ['day'+str(i) for i in range(1, 17)]
x_columns = [item for item in all_columns if item not in y_columns]

features_all = x_columns
features_all.remove("index") 
features_all.remove("date") 
features_all.remove("item_nbr") 
features_all.remove("store_nbr") 

X_train_out = train_out[x_columns]
X_val_out = val_out[x_columns]

y_train = train_out[y_columns].values
y_val = val_out[y_columns].values

X_train_allF = X_train_out[features_all]
X_val_allF = X_val_out[features_all]
X_test_allF = X_test_out[features_all]

#del train_out, val_out
#del X_train_out, X_val_out, X_test_out
gc.collect()

##########################################################################
logger.info('Training and predicting models...')

params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []


2018-01-03 11:54:17,440 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-03 11:54:17,440 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-03 11:54:17,440 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-03 11:54:17,440 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-03 11:54:17,440 __main__ 1 [INFO]    [<module>] Preparing traing dataset... 
2018-01-03 11:54:17,531 __main__ 29 [INFO]    [<module>] Training and predicting models... 
2018-01-03 11:54:17,531 __main__ 29 [INFO]    [<module>] Training and predicting models... 
2018-01-03 11:54:17,531 __main__ 29 [INFO]    [<module>] Training and predicting models... 
2018-01-03 11:54:17,531 __main__ 29 [INFO]    [<module>] Training and predicting models... 
2018-01-03 11:54:17,531 __main__ 29 [INFO]    [<module>] Training and predicting models... 


In [105]:
train_out["date"] = pd.to_datetime(train_out["date"])

In [106]:
train_out = train_out.loc[train_out["date"] < '2017-07-19', ]

In [103]:
train_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46163 entries, 0 to 46162
Data columns (total 91 columns):
index                  46163 non-null int64
date                   46163 non-null object
day_1_2017             46163 non-null float64
item_nbr               46163 non-null int64
mean_140_2017          46163 non-null float64
mean_14_2017           46163 non-null float64
mean_182_2017          46163 non-null float64
mean_21_2017           46163 non-null float64
mean_30_2017           46163 non-null float64
mean_364_2017          46163 non-null float64
mean_3_2017            46163 non-null float64
mean_42_2017           46163 non-null float64
mean_60_2017           46163 non-null float64
mean_7_2017            46163 non-null float64
mean_91_2017           46163 non-null float64
mean_ly_14_2017        46163 non-null float64
mean_ly_21_2017        46163 non-null float64
mean_ly_30_2017        46163 non-null float64
mean_ly_7_2017         46163 non-null float64
mean_ly_n16d_2017     

In [83]:
train_out[date < date(2017,7,19),]

NameError: name 'datetime' is not defined

In [7]:

train_week_2017 = 9
    
features_all = X_train_allF.columns.tolist()

for i in range(16):
    print("=" * 70)
    logger.info("Step %d" % (i+1))
    print("=" * 70)
    features_t = features_all.copy()

    for j in range(7):
        if j != i%7:
            features_t.remove('mean_4_dow{}_2017'.format(j))
            features_t.remove('mean_20_dow{}_2017'.format(j))
            features_t.remove('mean_52_dow{}_2017'.format(j))
            features_t.remove('mean_ly3w_dow{}_2017'.format(j))
            features_t.remove('mean_ly8w_dow{}_2017'.format(j))

    X_train = X_train_allF[features_t]
    X_val = X_val_allF[features_t]
    X_test = X_test_allF[features_t]

    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]]) * 0.25 + 1
    )



    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain], early_stopping_rounds=50, verbose_eval=100
    )

    logger.info("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))

    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))


    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))

#del X_train, y_train
#del dtrain
#gc.collect()

2018-01-03 10:37:56,116 __main__ 8 [INFO]    [<module>] Step 1 


/home/zyp/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1030: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.253313
[200]	training's l2: 0.237332
[300]	training's l2: 0.224244
[400]	training's l2: 0.212518


2018-01-03 10:38:00,115 __main__ 39 [INFO]    [<module>] mean_14_2017: 65963.74
mean_7_2017: 56151.99
mean_20_dow0_2017: 7203.99
mean_21_2017: 6732.14
mean_30_2017: 4899.73
promo_0: 4518.13
mean_4_dow0_2017: 3209.92
mean_3_2017: 3081.67
mean_52_dow0_2017: 3009.99
mean_91_2017: 1960.86
day_1_2017: 1808.26
mean_42_2017: 1564.58
mean_60_2017: 1443.28
mean_182_2017: 1141.14
mean_ly3w_dow0_2017: 1009.32
mean_ly_n16d_2017: 983.96
promo_14_2017: 964.05
mean_364_2017: 922.34
mean_ly8w_dow0_2017: 921.83
mean_140_2017: 887.87
mean_ly_7_2017: 848.34
mean_ly_14_2017: 790.11
mean_ly_30_2017: 776.04
promo_140_2017: 753.67
mean_ly_21_2017: 743.16
promo_60_2017: 527.88
promo_7: 247.19
promo_14: 129.04
promo_9: 87.95
promo_15: 73.13
promo_2: 64.42
promo_10: 40.11
promo_3: 35.30
promo_1: 25.75
promo_5: 21.48
promo_4: 16.04
promo_8: 15.10
promo_13: 13.94
promo_11: 13.43
promo_6: 10.85
promo_12: 7.89 


[500]	training's l2: 0.202376


...

2018-01-03 10:38:40,703 __main__ 8 [INFO]    [<module>] Step 16 


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.296819
[200]	training's l2: 0.277209
[300]	training's l2: 0.261781
[400]	training's l2: 0.248316


2018-01-03 10:38:43,356 __main__ 39 [INFO]    [<module>] mean_42_2017: 40183.06
mean_60_2017: 20926.77
mean_30_2017: 16678.11
mean_21_2017: 14761.16
mean_14_2017: 6517.70
mean_20_dow1_2017: 5751.20
promo_15: 3073.32
mean_4_dow1_2017: 2922.21
mean_52_dow1_2017: 2817.26
mean_7_2017: 1893.45
mean_182_2017: 1547.65
mean_3_2017: 1522.08
mean_91_2017: 1371.43
mean_ly_n16d_2017: 1229.06
mean_ly8w_dow1_2017: 1208.81
day_1_2017: 1136.48
mean_ly3w_dow1_2017: 1101.43
mean_364_2017: 1056.00
mean_140_2017: 1050.92
mean_ly_14_2017: 975.85
mean_ly_7_2017: 971.85
mean_ly_30_2017: 895.01
promo_140_2017: 839.96
mean_ly_21_2017: 832.27
promo_60_2017: 572.51
promo_14: 570.51
promo_14_2017: 508.03
promo_0: 116.50
promo_7: 100.77
promo_10: 71.18
promo_9: 64.29
promo_2: 50.72
promo_13: 49.99
promo_12: 36.39
promo_11: 32.87
promo_5: 29.06
promo_8: 27.71
promo_6: 22.12
promo_3: 14.96
promo_4: 12.26
promo_1: 11.86 


[500]	training's l2: 0.236348


187

In [32]:

logger.info('Making submission...')

X_test_out.reset_index(inplace = True)
del X_test_out["date"]
X_test_out = X_test_out.set_index(["store_nbr", "item_nbr"])

y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=X_test_out.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")


2018-01-03 10:54:06,461 __main__ 2 [INFO]    [<module>] Making submission... 
2018-01-03 10:54:06,461 __main__ 2 [INFO]    [<module>] Making submission... 
2018-01-03 10:54:06,461 __main__ 2 [INFO]    [<module>] Making submission... 


In [33]:
df_preds.tail(1)

,,,unit_sales
store_nbr,item_nbr,,
1,2127114,2017-08-31,0.061506


In [34]:
X_test_out.head(1)

,,index,day_1_2017,mean_140_2017,mean_14_2017,mean_182_2017,mean_21_2017,mean_30_2017,mean_364_2017,mean_3_2017,mean_42_2017,...,promo_6,promo_7,promo_8,promo_9,promo_10,promo_11,promo_12,promo_13,promo_14,promo_15
store_nbr,item_nbr,,,,,,,,,,,,,,,,,,,,,
1,96995,0,0.0,0.153952,0.334438,0.118424,0.275274,0.275522,0.059212,0.0,0.196801,...,0,0,0,0,0,0,0,0,0,0


In [37]:

df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)

submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('../submit/T040_tmp.csv', float_format='%.4f', index=None)

# PZ, Check overral result
print("SUM =",  submission.unit_sales.sum())
print("MEAN =",  submission.unit_sales.mean())

##########################################################################
df_prev = submission

df_sub = pd.read_csv('../input/sub_zero_30d.csv')

t_new = pd.merge(df_prev, df_sub, on=['id'], how='left')
t_new['unit_sales'] = t_new.unit_sales_y.combine_first(t_new.unit_sales_x)

submission = t_new[['id', 'unit_sales']]
del t_new

print("Merged  SUM =",  submission.unit_sales.sum())
print("Merged  MEAN =",  submission.unit_sales.mean())

submission.to_csv('../submit/T040_split.csv.gz',
                  float_format='%.4f', index=None, compression='gzip')


SUM = 3325.190447266003
MEAN = 0.043304080733275206
Merged  SUM = 3321.2397056780615
Merged  MEAN = 0.04325263007641999
